In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cv2
import numpy as np
import os.path
import requests
import statistics
from enum import IntEnum
from PIL import Image, ImageDraw

Some information about the games from the https://www.harmmade.com/vectorracer/#

* Map contains grid with the step equal to 15
* Tracks must be exactly 794 pixels wide and at least 599 pixels high
* The grid spacing is 15 pixels; the start point is used as an anchor point for the grid

In [ ]:
def download_image(img):
    if os.path.exists(img):
        return
    data = requests.get('https://www.harmmade.com/vectorracer/tracks/{}'.format(img)).content
    with open(img, 'wb') as handler:
        handler.write(data)
      
class Track:
    def __init__(self, image):
        self._orig = image
        self._classify_colors()
        self._start = self._find_start()
        self._canvas = self._orig.copy()
        self._track = self._fill_track()
        self._find_checkoints_on_image()
        self._grid_step = 15

    def save_canvas(self, name):
        self._canvas.save(name)
    
    def _classify_colors(self):
        self.TRACK = 20
        self.BOUNDARY = None
        self.CH = [[], [], [], [], []]
        self.DIR = []
        self.START = []
        self.FINISH = None
        self.EMPTY = None
        palette = self._orig.getpalette()
        for _, idx in self._orig.getcolors():
            cl = palette[3*idx], palette[3*idx+1], palette[3*idx+2]
            match cl:
                case (60, 120, 240):
                    self.BOUNDARY = idx
                case (104, 255, 104) | ( 80, 236, 98):
                    self.CH[0].append(idx)
                case (255, 104, 104) | (230,  86, 98):
                    self.CH[1].append(idx)
                case (255, 104, 255) | (230,  86, 248):
                    self.CH[2].append(idx)
                case (255, 179, 104) | (230, 161, 98):
                    self.CH[3].append(idx)
                case (195, 195, 195) | (149, 160, 183):
                    self.DIR.append(idx)
                case (155, 155, 155) | ( 36,  72, 145) | (118, 127, 145):
                    self.START.append(idx)
                case (255, 255, 255):
                    self.EMPTY = idx
                case ( 0, 0, 0):
                    self.FINISH = idx
                    self.CH[4].append(idx)
                case (195, 210, 240):
                    self.GRID = idx
    
    def _find_checkoints_on_image(self):
        self._checkpoints = [idx for (idx, arr) in enumerate(self.CH) if len(arr) > 0]
    
    def _find_start(self):
        xs = []
        ys = []
        for x in range(self._orig.width):
            for y in range(self._orig.height):
                v = self._orig.getpixel((x,y))
                if v in self.START:
                    xs.append(x)
                    ys.append(y)
        return statistics.median(xs), statistics.median(ys)

    def _fill_track(self):
        px = np.array(self._orig)
        px[px != self.BOUNDARY] = self.EMPTY
        cv2.floodFill(px, None, self._start, self.TRACK, 0, 1)
        px[px != self.TRACK] = self.EMPTY
        rv = Image.fromarray(px, mode='P')
        rv.putpalette(self._orig.getpalette())
        return rv
        
    def pt(self, pos):
        return (self._start[0] + self._grid_step * pos[0], self._start[1] - self._grid_step * pos[1])

    def is_valid(self, pos):
        p = self.pt(pos)
        return p[0] >= 0 and p[1] >= 0 and p[0] < self._orig.width and p[1] < self._orig.height
        
    def is_valid_pt(self, p):
        return p[0] >= 0 and p[1] >= 0 and p[0] < self._orig.width and p[1] < self._orig.height

    def is_on_track(self, pos):
        p = self.pt(pos)
        return self.is_valid(pos) and self._track.getpixel(p) == self.TRACK

    def is_on_track_pt(self, p):
        return self._track.getpixel(p) == self.TRACK

    def is_on_track_segment(self, pos0, pos1):
        if not self.is_on_track(pos0):
            return False
        for p in np.linspace(pos0, pos1, 10):
            if not self.is_on_track(p):
                return False
        return self.is_on_track(pos1)
    
    def next_checkpoint(self, current_checkoint):
        for i in range(current_checkoint + 1, len(self.CH)):
            if len(self.CH[i]) > 0:
                return i
        return None
    
    def find_next_target_checkoint(self, pos0, pos1):
        (_, _, _, _, current_checkoint) = pos0
        (x0,y0) = self.pt(pos0)
        prev_pt = (int(x0), int(y0))
        switches = []
        invalid = False
        idx = 0
        for pos in np.linspace(pos0, pos1, 100):
            idx = idx + 1
            (x,y) = self.pt(pos)
            pt = (int(x), int(y))
            if pt == prev_pt:
                continue
            prev_pt = pt
            if not self.is_valid_pt(pt):
                switches.append(pt)
                invalid = True
                break
            if not self.is_on_track_pt(pt):
                switches.append(pt)
                invalid = True
                break
            cl = self._orig.getpixel(pt)
            if cl in self.CH[current_checkoint]:
                switches.append(pt)
                current_checkoint = self.next_checkpoint(current_checkoint)
                if current_checkoint is None:
                    break
        return current_checkoint, switches, invalid, idx/100
    
    def draw(self, pos):
        if self.is_on_track(pos):
            d = 3
            fill = 6
        else:
            d = 10
            fill = 3
        img = ImageDraw.Draw(self._canvas)
        (px, py) = self.pt(pos)
        img.ellipse([(px-d, py-d), (px+d, py+d)], fill=fill, outline=30)
        
    def draw_segment(self, pos0, pos1):
        pos0 = (pos0[0], pos0[1], pos0[2], pos0[3])
        pos1 = (pos1[0], pos1[1], pos1[2], pos1[3])
        if self.is_on_track_segment(pos0, pos1):
            fill = 77
            width = 0
        else:
            fill = 3
            width = 3
        p0 = self.pt(pos0)
        p1 = self.pt(pos1)
        img = ImageDraw.Draw(self._canvas)
        img.line([p0, p1], fill=fill, width=width)
        
    def draw_trajectory(self, pts):
        for i in range(1, len(pts)):
            self.draw_segment(pts[i-1], pts[i])
        for pt in pts:
            self.draw(pt)
    
    def initial(self):
        return (0, 0, 0, 0, 0)
    
    def neighbour(self, pos, acceleration):
        (ddx, ddy) = acceleration
        (x, y, dx, dy, checkpoint) = pos
        s = (x + dx + ddx, y + dy + ddy, dx + ddx, dy + ddy, checkpoint)
        #if abs(s[2]) > 2 or abs(s[3]) > 2:
        #    return None
        if s == pos:
            return None
        next_checkpoint, at, invalid, idx = self.find_next_target_checkoint(pos, s)
        if next_checkpoint is None:
            return (s[0], s[1], s[2], s[3], next_checkpoint), (idx, at[-1])
        elif invalid:
            return None
        else:
            return (s[0], s[1], s[2], s[3], next_checkpoint), None

    def vals(self, dx):
        if dx == 0:
            return (0, 1, -1)
        elif dx > 0:
            return (-1, 0, 1)
        else:
            return (1, 0, -1)
        
    def neighbours(self, pos):
        rv = []
        for ddx in self.vals(pos[2]):
            for ddy in self.vals(pos[3]):
                neighbour = self.neighbour(pos, (ddx, ddy))
                if not neighbour is None:
                    rv.append(neighbour)
        return rv    

In [ ]:
download_image('oval.png')
track = Track(Image.open('oval.png'))

In [ ]:
def bfs(tr):
    initial = track.initial()
    states = [initial]
    idx = 0
    distances = {initial: 0}
    came_from = {}
    finishes = []
    found_distance = None
    last_dst = -1
    while idx < len(states):
        current = states[idx]
        dist = distances[current]
        if dist > last_dst:
            last_dst = dist
            print("Processing... dist={} remaining={}".format(dist, len(states) - idx))
        idx = idx + 1
        if not found_distance is None and dist >= found_distance:
            break
        for (neighbour, info) in track.neighbours(current):
            if neighbour in distances:
                continue
            came_from[neighbour] = current
            if neighbour[-1] is None:
                finishes.append((neighbour, info))
                found_distance = dist + 1
            else:
                distances[neighbour] = dist + 1
                states.append(neighbour)
    return finishes, came_from

In [ ]:
%%time

finishes, came_from = bfs(track)

In [ ]:
def trajectory(finish, came_from):
    tr = [finish]
    while tr[-1] in came_from:
        tr.append(came_from[tr[-1]])
    return list(reversed(tr))

def trajectory_length(tr):
    d = 0
    for i in range(1, len(tr)):
        p1 = tr[i-1]
        p2 = tr[i]
        d = d +(p2[0]-p1[0])*(p2[0]-p1[0]) + (p2[1]-p1[1])*(p2[1]-p1[1])
    return d

def best_finish(finishes):
    s = sorted((t, s[2]*s[2] + s[3]*s[3], s) for (s, (t, _)) in finishes)
    best = s[0][0]
    rv = [trajectory(x[2], came_from) for x in s if x[0]==best]
    lengths = [(trajectory_length(x), x) for x in rv]
    return min(lengths)[1]

tr = best_finish(finishes)
track.draw_trajectory(tr)

#track.save_canvas('oval-best.png')
track.draw((-25, 25))
track._canvas